<a href="https://colab.research.google.com/github/bcandraas/Data-Analyst-Portfolio/blob/main/vending_machine_sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vending Machine Sales Dataset

This dataset represents vending machine data from various locations in Central New Jersey. The locations include a library, a mall, office location and a manufacturing locations. Data analyst can make use of the data to understand trends, user behavior and overall preferences by consumers at different locations.

## Import Library and Dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from datetime import timedelta
from sklearn import cluster
from google.colab import files

In [ ]:
sheet_url = 'https://docs.google.com/spreadsheets/d/1mlVEdnDS0uIk8HhJdXHLVP-Dmow62H50GG5cEdZLRk4/edit#gid=323701123'
sheet_url_trf = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
print(sheet_url_trf)
df = pd.read_csv(sheet_url_trf)
df.head()

https://docs.google.com/spreadsheets/d/1mlVEdnDS0uIk8HhJdXHLVP-Dmow62H50GG5cEdZLRk4/export?format=csv&gid=323701123


,Status,Device ID,Location,Machine,Product,Category,Transaction,TransDate,Type,RCoil,RPrice,RQty,MCoil,MPrice,MQty,LineTotal,TransTotal,Prcd Date
0,Processed,VJ300320611,Brunswick Sq Mall,BSQ Mall x1366 - ATT,Red Bull - Energy Drink - Sugar Free,Carbonated,14515778905,"Saturday, January 1, 2022",Credit,148,3.5,1,148,3.5,1,3.5,3.5,1/1/2022
1,Processed,VJ300320611,Brunswick Sq Mall,BSQ Mall x1366 - ATT,Red Bull - Energy Drink - Sugar Free,Carbonated,14516018629,"Saturday, January 1, 2022",Credit,148,3.5,1,148,3.5,1,3.5,5.0,1/1/2022
2,Processed,VJ300320611,Brunswick Sq Mall,BSQ Mall x1366 - ATT,Takis - Hot Chilli Pepper & Lime,Food,14516018629,"Saturday, January 1, 2022",Credit,123,1.5,1,123,1.5,1,1.5,5.0,1/1/2022
3,Processed,VJ300320611,Brunswick Sq Mall,BSQ Mall x1366 - ATT,Takis - Hot Chilli Pepper & Lime,Food,14516020373,"Saturday, January 1, 2022",Credit,123,1.5,1,123,1.5,1,1.5,1.5,1/1/2022
4,Processed,VJ300320611,Brunswick Sq Mall,BSQ Mall x1366 - ATT,Red Bull - Energy Drink - Sugar Free,Carbonated,14516021756,"Saturday, January 1, 2022",Credit,148,3.5,1,148,3.5,1,3.5,3.5,1/1/2022


## Handling Missing Data

In [ ]:
df.isna().sum()

Status           0
Device ID        0
Location         0
Machine          0
Product          4
Category       185
Transaction      0
TransDate        0
Type             0
RCoil            0
RPrice           0
RQty             0
MCoil            0
MPrice           1
MQty             0
LineTotal        0
TransTotal       0
Prcd Date        0
dtype: int64

In [ ]:
df.shape

(6445, 18)

In [ ]:
dfcheck = df[df['Category'].isnull()]
dfcheck.head()

,Status,Device ID,Location,Machine,Product,Category,Transaction,TransDate,Type,RCoil,RPrice,RQty,MCoil,MPrice,MQty,LineTotal,TransTotal,Prcd Date
1232,Processed,VJ300320692,EB Public Library,EB Public Library x1380,NaN,NaN,14835073327,"Wednesday, March 16, 2022",Credit,120,2.0,1,120,2.0,1,2.0,2.00,3/30/2022
1431,Processed,VJ300320692,EB Public Library,EB Public Library x1380,NaN,NaN,14873793590,"Thursday, March 24, 2022",Cash,120,2.0,1,120,2.0,1,2.0,2.00,3/30/2022
1439,Processed,VJ300320692,EB Public Library,EB Public Library x1380,NaN,NaN,14878665903,"Friday, March 25, 2022",Credit,120,2.0,1,120,2.0,1,2.0,3.25,3/30/2022
1448,Processed,VJ300320609,GuttenPlans,GuttenPlans x1367,Doritos Dinamita Chile Lemon,NaN,14879782611,"Friday, March 25, 2022",Cash,125,1.0,1,125,1.0,1,1.0,1.00,3/25/2022
1453,Processed,VJ300320609,GuttenPlans,GuttenPlans x1367,Doritos Dinamita Chile Lemon,NaN,14880181107,"Saturday, March 26, 2022",Cash,113,1.5,1,113,1.5,1,1.5,1.50,3/25/2022


In [ ]:
dfcheck['Product'].unique()

array([nan, 'Doritos Dinamita Chile Lemon', 'Doritos Spicy Nacho',
       'Mini Chips Ahoy - Go Paks', 'Oreo Mini - Go Paks',
       'Teddy Grahams - Go Paks', 'Starbucks Doubleshot Energy - Coffee',
       'Canada Dry - Ginger Ale & Lemonde', 'Canada Dry - Ginger Ale'],
      dtype=object)

In [ ]:
df['Category'] = np.where((df['Product'] == 'Doritos Dinamita Chile Lemon'), 'Food', df['Category'])
df['Category'] = np.where((df['Product'] == 'Doritos Spicy Nacho'), 'Food', df['Category'])
df['Category'] = np.where((df['Product'] == 'Mini Chips Ahoy - Go Paks'), 'Food', df['Category'])
df['Category'] = np.where((df['Product'] == 'Oreo Mini - Go Paks'), 'Food', df['Category'])
df['Category'] = np.where((df['Product'] == 'Teddy Grahams - Go Paks'), 'Food', df['Category'])
df['Category'] = np.where((df['Product'] == 'Starbucks Doubleshot Energy - Coffee'), 'Non Carbonated', df['Category'])
df['Category'] = np.where((df['Product'] == 'Canada Dry - Ginger Ale & Lemonde'), 'Carbonated', df['Category'])
df['Category'] = np.where((df['Product'] == 'Canada Dry - Ginger Ale'), 'Carbonated', df['Category'])

In [ ]:
df.isna().sum()

Status         0
Device ID      0
Location       0
Machine        0
Product        4
Category       4
Transaction    0
TransDate      0
Type           0
RCoil          0
RPrice         0
RQty           0
MCoil          0
MPrice         1
MQty           0
LineTotal      0
TransTotal     0
Prcd Date      0
dtype: int64

In [ ]:
df.dropna(axis=0,inplace=True)
df.isna().sum()

Status         0
Device ID      0
Location       0
Machine        0
Product        0
Category       0
Transaction    0
TransDate      0
Type           0
RCoil          0
RPrice         0
RQty           0
MCoil          0
MPrice         0
MQty           0
LineTotal      0
TransTotal     0
Prcd Date      0
dtype: int64

In [ ]:
df.shape

(6441, 18)

## Handling Duplicate

In [ ]:
df.duplicated().sum()

0

## Export Data to CSV

After this, dataset will be transfered to Tableau to create the Dashboard

In [ ]:
df.to_csv('vending_machine_sales.csv')